# Clean StatFox Matchup Scrapes
#### `Jonathan Sims 2020-03-13`
- USE: Get semi-parsed html for each statfox matchup page and clean into modeling input
    - Treat each matchup page as one obs in RF model to asses feature importance
    - Matchup data is pre-computed as of day of match, but needs lots of cleaning 
    - Includes opening and close money and total lines

In [54]:
### Set datetime stamp in YYYYMMDD for all file outputs

from datetime import date as dt
YMD = dt.today().isoformat().replace('-','') + '.'

In [41]:
### Import all modules at top

import sys
import math
import boto3
import numpy as np
import pandas as pd 
import pickle
import codecs
import os.path
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.request import urlopen
from sklearn.impute import SimpleImputer

In [42]:
def FromPickleS3(bucketname, keyname):
    """Get dataframe from s3 and unpickle
    """
    s3 = boto3.client('s3')
    pkldf = s3.get_object(Bucket=bucketname,Key=keyname)['Body'].read()
    df = pickle.loads(pkldf)
    return df

In [43]:
def Flatten2DLabel(obj, row_label, col_label):
    """Flattens a dataframe to single row and concatenates row and 
    column lables within the dataframe into one column name.                                 
    """
    # Return all row indexes after last row_label
    st_row = row_label[-1]+1
    # Return all columns after last col_label
    st_col = col_label[-1]+1


    # Get row labels and concatenate
    for x in row_label:
        if x == row_label[0]:
            cols = []
            cols = obj.iloc[x][st_col:]
        else:
            cols = cols+'_'+obj.iloc[x][st_col:]

    # Get column labels and concatenate
    for x  in col_label:
        if x == col_label[0]:
            rows = []
            rows = obj[st_row:][x]
        else:
            rows = rows+'_'+obj[st_row:][x]  

    # Strip non alphanumeric
    colstrip = cols.str.replace('[^\w]','')
    rowstrip = rows.str.replace('[^\w]','')


    obj_in = pd.DataFrame([])
    for x in rowstrip.index:
        for y in colstrip.index:
            obj_tmp = pd.DataFrame([obj.iloc[x][y]], columns=[str(colstrip[y])+'_'+str(rowstrip[x])])
            obj_in = pd.concat([obj_in,obj_tmp], axis=1)
            
    return obj_in

In [44]:
def CleanStatfoxScrape(obj):
    """Takes flattened sub-tables from statfox matchup
    page and cleans and shortens feature names
    """
    # Team names to V or H
    obj.columns = obj.columns.str.replace(tm_v,'V_')
    obj.columns = obj.columns.str.replace(tm_h,'H_')

    # Shorten sub-table names
    obj.columns = obj.columns.str.replace('CurrentSeasonPerformance', 'Overall')
    obj.columns = obj.columns.str.replace('TeamHittingandFieldingStatistics','HitField')
    obj.columns = obj.columns.str.replace('BullpenPitchingStatistics','Bullpen')
    
    return obj

In [45]:
def DuplicatedVarnames(df):
    """Return a dict of all variable names that 
    are duplicated in a given dataframe."""
    repeat_dict = {}
    var_list = list(df) # list of varnames as strings
    for varname in var_list:
        # make a list of all instances of that varname
        test_list = [v for v in var_list if v == varname] 
        # if more than one instance, report duplications in repeat_dict
        if len(test_list) > 1: 
            repeat_dict[varname] = len(test_list)
    return repeat_dict

### Get list of games

In [48]:
glhead = pd.read_csv('GLHEADER.CSV',header=None)

gms = pd.read_csv('20200313.matchup_list.2010-2018.csv.gz',
                  header=0, 
                  names=glhead[0], 
                  low_memory=False
                 )

gms = gms[['date','team_h','team_v','score_h','score_v']]

teams = pd.read_csv('TEAM_NAMES.CSV',
                    header=0,
                    index_col=['name1'],
                    usecols=['name1','name3']
                   )

teams = teams['name3'].to_dict()

gms['team_h'] = gms['team_h'].map(lambda x: teams[x.upper()])
gms['team_v'] = gms['team_v'].map(lambda x: teams[x.upper()])

### Get dataframe from s3 and unpickle
### Format and append each matchup together from s3

In [50]:
for x in gms.index[:2]:
    print(gms.iloc[x])

date        20100405
team_h     LA ANGELS
team_v     MINNESOTA
score_h            6
score_v            3
Name: 0, dtype: object
date            20100405
team_h     CHI WHITE SOX
team_v         CLEVELAND
score_h                6
score_v                0
Name: 1, dtype: object


In [10]:
df_all = pd.DataFrame()


# for x in gms.index[20600:]:
    try:
        
        ### Parse date, team names, and score from games list
        
        dt = str(gms.loc[x,'date'])
        tm_h = str(gms.loc[x,'team_h']).replace(' ','')                                                              
        tm_v = str(gms.loc[x,'team_v']).replace(' ','')
        sc_h = gms.loc[x,'score_h']
        sc_v = gms.loc[x,'score_v']

        ### Adjust URL if second game of double header
        
        if (x > 1) and (str(gms.loc[x-1,'date']) == dt) and (str(gms.loc[x-1,'team_h']).replace(' ','') == tm_h):
            tm_h_url = tm_h+'2'
        else:
            tm_h_url = tm_h

        ### Create dataframe from s3 pickle
        
        df = FromPickleS3(bucketname='scrapes-rawhtml-dev', keyname='statfox_DEV/'+dt+tm_h_url+'.pkl')

        ### Show progress for my OCD
        
        if x%100 == 0:
            timestamp = str(datetime.now())
            print(timestamp, ' - ', 'statfox_DEV/'+dt+tm_h_url+'.pkl', ' - ',x)

        ### Parse and concatenate all features together, by matchup

        # 6:  Overall - board and line
        # 11: Away - Current Season Performance
        # 12: Away - Team Hitting and Fielding
        # 13: Away - Bullpen Pitching 
        # 14: Home - Current Season Performance
        # 15: Home - Team Hitting and Fielding
        # 16: Home - Bullpen Pitching 
        df6  = df[6] 
        df11 = df[11]
        df12 = df[12]
        df13 = df[13]
        df14 = df[14]
        df15 = df[15]
        df16 = df[16]

        df6 = df6.transpose() # Make board and team labels first
        df6_wd = Flatten2DLabel(df6, [1], [0,1])

        df11_wd = Flatten2DLabel(df11, [0,1,2], [0])
        df14_wd = Flatten2DLabel(df14, [0,1,2], [0])

        df12_wd = Flatten2DLabel(df12, [0,1,2], [0])
        df15_wd = Flatten2DLabel(df15, [0,1,2], [0])

        df13_wd = Flatten2DLabel(df13, [0,1], [0])
        df16_wd = Flatten2DLabel(df16, [0,1], [0])

        ### Concatenate all sub-tables and clean labels

        df_wd_all = pd.concat([df6_wd, df11_wd, df12_wd, df13_wd, df14_wd, df15_wd, df16_wd], axis=1)
        df_wd_all = CleanStatfoxScrape(df_wd_all)

        ### Add matchup index value date-board_h-board_v

        board_h = df6[3].iloc[0]
        board_v = df6[2].iloc[0]
        df_wd_all.index = [int(dt)*1000000+int(board_h)*1000+int(board_v)]

        ### Add team names and final scores

        df_wd_all['tm_h'] = tm_h
        df_wd_all['tm_v'] = tm_v
        df_wd_all['sc_h'] = sc_h
        df_wd_all['sc_v'] = sc_v

        ### Drop columns duplicates (usually nan)

        for x in DuplicatedVarnames(df_wd_all):
            df_wd_all = df_wd_all.drop(list(df_wd_all.filter(regex=x)), axis=1)

        ### Split features on hyphen if OU (over-under) or WL (win-loss)        

        ou_cols = [col for col in df_wd_all.columns if 'OU' in col]

        if ou_cols != []:
            for x in ou_cols:
                xO = x.replace('OU', 'O')
                xU = x.replace('OU', 'U')
                df_wd_all[[xO,xU]] = df_wd_all[x].str.split("-",expand=True)    
                df_wd_all = df_wd_all.drop(x, axis=1)

        wl_cols = [col for col in df_wd_all.columns if 'WL' in col]

        if wl_cols != []:
            for x in wl_cols:
                xW = x.replace('WL', 'W')
                xL = x.replace('WL', 'L')
                df_wd_all[[xW,xL]] = df_wd_all[x].str.split("-",expand=True)    
                df_wd_all = df_wd_all.drop(x, axis=1)    

        ### Create target variable (home win)

        df_wd_all['win_h'] = np.where(df_wd_all['sc_h'] > df_wd_all['sc_v'], 1, 0)

        ### Combine feature rows

        df_all = pd.concat([df_all, df_wd_all], ignore_index=False, sort=True)
    
    ### Print exception to log and go to next iteration (URL)
    
    except Exception as exc:
        timestamp = str(datetime.now())
        exc = str(exc)
        excstamp = timestamp+' - '+exc+' - '+dt+tm_h_url
        print(excstamp, file=open(file_l, 'a'))
        continue
    
    except IndexError as err:
        timestamp = str(datetime.now())
        excstamp = timestamp+' - '+err+' - '+dt+tm_h_url
        print(excstamp, file=open(file_l, 'a'))
        continue

2020-03-12 03:13:28.735535  -  statfox_DEV/20180625KANSASCITY.pkl  -  20600
2020-03-12 03:14:24.561005  -  statfox_DEV/20180703COLORADO.pkl  -  20700
2020-03-12 03:15:22.653832  -  statfox_DEV/20180710LAANGELS.pkl  -  20800
2020-03-12 03:16:23.348677  -  statfox_DEV/20180721LAANGELS.pkl  -  20900
2020-03-12 03:17:26.065971  -  statfox_DEV/20180728BOSTON.pkl  -  21000
2020-03-12 03:18:31.321667  -  statfox_DEV/20180805LADODGERS.pkl  -  21100
2020-03-12 03:19:39.274568  -  statfox_DEV/20180812LAANGELS.pkl  -  21200
2020-03-12 03:20:52.291550  -  statfox_DEV/20180819OAKLAND.pkl  -  21300
2020-03-12 03:22:06.514873  -  statfox_DEV/20180827SANFRANCISCO.pkl  -  21400
2020-03-12 03:23:23.910832  -  statfox_DEV/20180903OAKLAND.pkl  -  21500
2020-03-12 03:24:43.069145  -  statfox_DEV/20180911BOSTON.pkl  -  21600
2020-03-12 03:26:06.577939  -  statfox_DEV/20180918OAKLAND.pkl  -  21700
2020-03-12 03:27:34.141391  -  statfox_DEV/20180926SANFRANCISCO.pkl  -  21800


### Save cleaned to gzip tsv

In [11]:
df_all.shape

(1267, 581)

In [12]:
df_all.to_csv(YMD+'statfox_clean_scrapes.2010-2018.csv.gz', sep='\t')

In [56]:
file_in1 = '20200310.skr_statfox.2010-2018.0.7400.tsv.gz'
file_in2 = '20200310.skr_statfox.2010-2018.7400.10000.tsv.gz'
file_in3 = '20200310.skr_statfox.2010-2018.10000.12000.tsv.gz'
file_in4 = '20200310.skr_statfox.2010-2018.12000.16000.tsv.gz'
file_in5 = '20200310.skr_statfox.2010-2018.16000.20600.tsv.gz'
file_in6 = '20200310.skr_statfox.2010-2018.20600.tsv.gz'

### Import features, moneylines, target

In [57]:
### Import chunks and append

df1 = pd.read_csv(file_in1, sep='\t', low_memory=False)
df2 = pd.read_csv(file_in2, sep='\t', low_memory=False)
df3 = pd.read_csv(file_in3, sep='\t', low_memory=False)
df4 = pd.read_csv(file_in4, sep='\t', low_memory=False)
df5 = pd.read_csv(file_in5, sep='\t', low_memory=False)
df6 = pd.read_csv(file_in6, sep='\t', low_memory=False)

df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=False, sort=True)

In [58]:
### Clear useless dataframes

df1 = []
df2 = []
df3 = []
df4 = []
df5 = []
df6 = []

In [59]:
### Give proper name to the match index

df['matchidx'] = df['Unnamed: 0']
df = df.drop(['Unnamed: 0'], axis=1)

In [60]:
df.shape

(17632, 590)

In [61]:
df.tail()

,H_Bullpen_BB_AllGames,H_Bullpen_BB_HomeGames,H_Bullpen_BSV_AllGames,H_Bullpen_BSV_HomeGames,H_Bullpen_ERA_AllGames,H_Bullpen_ERA_HomeGames,H_Bullpen_ER_AllGames,H_Bullpen_ER_HomeGames,H_Bullpen_HR_AllGames,H_Bullpen_HR_HomeGames,...,V__Latest_Line,V__Latest_Total,V__Opening_Line,V__Opening_Total,sc_h,sc_v,tm_h,tm_v,win_h,matchidx
1262,221,105,27,10,4.51,4.22,303,142,89,39,...,+120,"Ov 9,-105",+135,"Ov 9,-110",5,4,MINNESOTA,CHIWHITESOX,1,20180930926925
1263,174,94,21,12,3.79,3.58,233,114,67,42,...,+155,"Ov 8.5,-110",+155,"Ov 8.5,-120",3,1,SEATTLE,TEXAS,1,20180930922921
1264,285,139,18,7,3.81,3.25,347,156,88,44,...,+150,"Ov 8,-115",+150,"Ov 8,-115",9,4,TAMPABAY,TORONTO,1,20180930924923
1265,273,137,18,10,3.34,3.76,217,121,51,29,...,+115,"Ov 8,-115",+115,"Ov 8,-115",1,3,CHICAGOCUBS,MILWAUKEE,0,20181001952951
1266,198,85,26,11,3.70,3.77,238,117,73,39,...,+150,"Ov 7,-105",+155,"Ov 7,-105",5,2,LADODGERS,COLORADO,1,20181001954953


### Clean up OU Totals, OSB, and DP features
- This should really be in skr_statfox_matchups.ipynb 

In [62]:
### Split by a comma then drop Ov and Un text

ou_cols = [col for col in df.columns if '_Total' in col]

if ou_cols != []:
    for x in ou_cols:
        xO = x.replace('_Total', '_Tot')
        xU = x.replace('_Total', '_TotLn')
        
        df[[xO,xU]] = df[x].str.split(",", expand=True)    
        df[xO] = df[xO].str.replace("Ov ","")
        df[xO] = df[xO].str.replace("Un ","")
        df = df.drop(x, axis=1)
    
new_ou_cols = [col for col in df.columns if '_Tot' in col]    
df[new_ou_cols].head()

,H__Latest_Tot,H__Latest_TotLn,H__Opening_Tot,H__Opening_TotLn,V__Latest_Tot,V__Latest_TotLn,V__Opening_Tot,V__Opening_TotLn
0,8.5,-105,8.5,-105,8.5,-115,8.5,-115
1,9,-115,9,-115,9,-105,9,-105
2,7,-110,7,-110,7,-110,7,-110
3,7,+105,7,+105,7,-125,7,-125
4,9,-115,9.5,-120,9,-105,9.5,+100


In [63]:
### Remove parantheses from numeric values

osb_cols = [col for col in df.columns if '_OSB_' in col]
dp_cols = [col for col in df.columns if '_DP_' in col]
paren_cols = osb_cols + dp_cols

if paren_cols != []:
    for x in paren_cols:
        df[x] = df[x].str.replace("(","")
        df[x] = df[x].str.replace(")","")
        
df[paren_cols].head()        

,H_HitField_TeamFielding_OSB_AllGames,H_HitField_TeamFielding_OSB_HomeGames,H_HitField_TeamFielding_OSB_LeftyStarters,H_HitField_TeamFielding_OSB_RightyStarters,V_HitField_TeamFielding_OSB_AllGames,V_HitField_TeamFielding_OSB_LeftyStarters,V_HitField_TeamFielding_OSB_RightyStarters,V_HitField_TeamFielding_OSB_RoadGames,H_HitField_TeamFielding_DP_AllGames,H_HitField_TeamFielding_DP_HomeGames,H_HitField_TeamFielding_DP_LeftyStarters,H_HitField_TeamFielding_DP_RightyStarters,V_HitField_TeamFielding_DP_AllGames,V_HitField_TeamFielding_DP_LeftyStarters,V_HitField_TeamFielding_DP_RightyStarters,V_HitField_TeamFielding_DP_RoadGames
0,129,64,NaN,90,104,NaN,70,37,182,94,NaN,114,138,NaN,77,67
1,124,66,NaN,88,111,31,NaN,58,157,77,NaN,103,170,32,NaN,82
2,108,47,NaN,78,86,NaN,61,45,160,98,NaN,96,164,NaN,112,88
3,86,40,NaN,59,65,NaN,46,26,155,78,NaN,104,150,NaN,98,68
4,97,50,NaN,62,87,NaN,58,48,168,97,NaN,109,168,NaN,112,83


In [64]:
### Remove percent signs

perc = [col for col in df.columns if '_Pct_' in col]

if perc != []:
    for x in perc:
        df[x] = df[x].str.replace("%","")
        df[x] = df[x].str.replace(" ","")

df[perc].head()

,H_Bullpen_Pct_AllGames,H_Bullpen_Pct_HomeGames,V_Bullpen_Pct_AllGames,V_Bullpen_Pct_RoadGames
0,76.1,73.5,73.8,79.3
1,66.7,68,58.1,53.6
2,60.7,57.7,65.6,58.1
3,76,77.3,63.6,60
4,78.9,82.8,62.5,68.4


### Separate features, targets, and misc

In [65]:
lines = [col for col in df.columns if 'Latest_Line' in col]
teams = ['tm_h', 'tm_v']
scores = ['sc_h','sc_v']
feats = ['win_h']
# drop = pd.concat([lines, totals, scores, feats])
drop = lines+scores+feats
drop

['H__Latest_Line', 'V__Latest_Line', 'sc_h', 'sc_v', 'win_h']

In [66]:
df_feat = df.drop(drop, axis=1).reset_index(drop=True)
df_targ = pd.to_numeric(df['win_h'], errors='coerce').reset_index(drop=True)
df_openline = pd.to_numeric(df['H__Opening_Line'], errors='coerce').reset_index(drop=True)
df_lateline = pd.to_numeric(df['H__Latest_Line'], errors='coerce').reset_index(drop=True)

### Add year and month variable

In [67]:
def get_month(x):
    """Take match index YYYYMMDDHHHVVV and return the month of game
    """
    flr = math.floor(x/100000000)
    flrmod = flr%100
    return str(flrmod)

def get_year(x):
    """Take match index YYYYMMDDHHHVVV and return the year of game
    """
    flr = math.floor(x/10000000000)
    return str(flr)

In [68]:
df_feat['month'] = df_feat['matchidx'].apply(get_month)
df_feat['year'] = df_feat['matchidx'].apply(get_year)

In [69]:
df_feat.shape

(17632, 591)

In [70]:
df_feat[['matchidx','year','month']].head()

,matchidx,year,month
0,20100405924923,2010,4
1,20100405918917,2010,4
2,20100405922921,2010,4
3,20100405926925,2010,4
4,20100405920919,2010,4


### Final Clean
- Try to convert object to numeric
- If except: convert object to binary dummies

In [71]:
def preprocess_tonumeric(x, coercecols):
    """Clean features to get everything numeric.
    1) Strip percent sign, space, and commas
    2) Convert objects to numeric if possible
    3) Else, convert objects to dummies
    """
#     output = pd.DataFrame()
    
    for col in coercecols:
        if col not in x.columns:
            raise ValueError
    
    for col, col_data in x.iteritems():
        
        if col_data.dtype == object and col in coercecols:
            
            col_data = pd.to_numeric(col_data, errors='coerce')
            x = x.drop(col, axis=1)
            
        elif col_data.dtype == object:
                
            try:
                col_data = pd.to_numeric(col_data)
                x = x.drop(col, axis=1)
                
            except:
                col_data = pd.get_dummies(col_data, prefix=col)
                x = x.drop(col, axis=1)
            
        x = pd.concat([x, col_data], axis=1)
    
    ### Keep first of all columns then drop duplicates

    Cols = list(x.columns)
    for i,item in enumerate(x.columns):
        if item in x.columns[:i]: Cols[i] = "toDROP"
    x.columns = Cols
    x = x.drop("toDROP",1)
    
    return x

### Impute missing values and create dummy for missing

In [72]:
# def preprocess_nan(x, fill_strategy='mean'):
def preprocess_nan(z):
    """Process NaNs in a dataframe to prepare for RF or other model.
    1) Creates dummy columns for each existing column with > 1 NaN
    2) Fill NaN in existing column with desired strategy (see scikit SimpleImputer)
    """
    for col in z.columns:
        if z[col].isnull().any().any():
            nancol = col+'_NaN'
            z[nancol] = [1 if s == True else 0 for s in z[col].isnull()]
        
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    x_out = pd.DataFrame(imp.fit_transform(z), columns=z.columns)
    
    return x_out

In [73]:
df_feat_num = preprocess_tonumeric(df_feat, coercecols=['H__Opening_Line', 'V__Opening_Line'])

In [74]:
df_feat_num[:2]

,H_Bullpen_BB_AllGames,H_Bullpen_BB_HomeGames,H_Bullpen_BSV_AllGames,H_Bullpen_BSV_HomeGames,H_Bullpen_ERA_AllGames,H_Bullpen_ERA_HomeGames,H_Bullpen_ER_AllGames,H_Bullpen_ER_HomeGames,H_Bullpen_HR_AllGames,H_Bullpen_HR_HomeGames,...,H__Latest_Tot,H__Latest_TotLn,H__Opening_Tot,H__Opening_TotLn,V__Latest_Tot,V__Latest_TotLn,V__Opening_Tot,V__Opening_TotLn,month,year
0,228,117,17,9,4.42,4.69,253,138,56,30,...,8.5,-105.0,8.5,-105.0,8.5,-115.0,8.5,-115.0,4,2010
1,190,107,18,8,4.06,4.41,212,126,49,25,...,9.0,-115.0,9.0,-115.0,9.0,-105.0,9.0,-105.0,4,2010


In [75]:
df_feat_num.shape

(17632, 651)

In [76]:
df.shape

(17632, 594)

In [77]:
df_feat_fill = preprocess_nan(df_feat_num)
df_feat_fill[:2]

,H_Bullpen_BB_AllGames,H_Bullpen_BB_HomeGames,H_Bullpen_BSV_AllGames,H_Bullpen_BSV_HomeGames,H_Bullpen_ERA_AllGames,H_Bullpen_ERA_HomeGames,H_Bullpen_ER_AllGames,H_Bullpen_ER_HomeGames,H_Bullpen_HR_AllGames,H_Bullpen_HR_HomeGames,...,V_HitField_TeamFielding_OSB_RoadGames_NaN,V__Opening_Line_NaN,H__Latest_Tot_NaN,H__Latest_TotLn_NaN,H__Opening_Tot_NaN,H__Opening_TotLn_NaN,V__Latest_Tot_NaN,V__Latest_TotLn_NaN,V__Opening_Tot_NaN,V__Opening_TotLn_NaN
0,228.0,117.0,17.0,9.0,4.42,4.69,253.0,138.0,56.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,190.0,107.0,18.0,8.0,4.06,4.41,212.0,126.0,49.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert open and close moneylines to probability

In [78]:
def american_to_probability(x):
    """Turns American +/- odds into probability 0 to 1 exclusive
    NOTE: Returns 0 if missing since 0 and 1 are impossible from lines
    """
    if x < -99:
        num = abs(x)
        pr = num/(100+num)
        return pr

    elif x >= 100:
        num = x
        pr = 100/(100+num)
        return pr
    
    elif x == None:
        pr = 0

#     else:
#         print('Error: No sign found in betting line string')

### Dedupe on matchup index (YYYYMMDDHHHVVV)

In [79]:
df_feat_fill = df_feat_fill.drop_duplicates(['matchidx'])

In [80]:
df_feat_fill.shape

(17573, 1119)

In [81]:
df_lateline.shape

(17632,)

In [82]:
df_lateline.head()

0   -130.0
1   -160.0
2   -110.0
3    120.0
4   -140.0
Name: H__Latest_Line, dtype: float64

### Convert moneylines to probabilities

In [83]:
### Separate open/close moneylines from features

df_lateline_prob = df_lateline.apply(american_to_probability)
df_lateline_prob = df_lateline_prob[lambda x: (x > 0) & (x < 1)]
df_openline_prob = df_openline.apply(american_to_probability)
df_openline_prob = df_openline_prob[lambda x: (x > 0) & (x < 1)]

In [84]:
### Convert line features to probability

lines = ['H__Opening_Line', 'V__Opening_Line', 'H__Opening_TotLn', 'V__Opening_TotLn']

for col in lines:
    newcol = col + 'Pr'
    df_feat_fill[newcol] = df_feat_fill[col].apply(american_to_probability)

In [85]:
### Check for any Opening lines (should have probabilities created *Pr*)

df_feat_fill[[col for col in df_feat_fill.columns if 'Opening' in col]].dtypes

H__Opening_Line         float64
V__Opening_Line         float64
H__Opening_Tot          float64
H__Opening_TotLn        float64
V__Opening_Tot          float64
V__Opening_TotLn        float64
H__Opening_Line_NaN     float64
V__Opening_Line_NaN     float64
H__Opening_Tot_NaN      float64
H__Opening_TotLn_NaN    float64
V__Opening_Tot_NaN      float64
V__Opening_TotLn_NaN    float64
H__Opening_LinePr       float64
V__Opening_LinePr       float64
H__Opening_TotLnPr      float64
V__Opening_TotLnPr      float64
dtype: object

In [86]:
### Check for any Closing lines (should be none)

df_feat_fill[[col for col in df_feat_fill.columns if 'Closing' in col]].dtypes

Series([], dtype: object)

In [87]:
### Check of any object cols (should be none)

for col, col_data in df_feat_fill.iteritems():
    if col_data.dtype == object:
        print(col)

### Export to tsv.gz then s3

In [88]:
df_feat_fill.to_csv(YMD+'statfox_features.tsv.gz', 
                    sep='\t')

In [93]:
s3 = boto3.client('s3')

s3.upload_file(YMD+'statfox_features.tsv.gz', 
               Bucket='scrapes-rawhtml-dev', 
               Key='testfolder/'+YMD+'statfox_features.csv.gz')

In [95]:
s3.download_file?

Signature: s3.download_file(Bucket, Key, Filename, ExtraArgs=None, Callback=None, Config=None)
Docstring:
Download an S3 object to a file.

Usage::

    import boto3
    s3 = boto3.resource('s3')
    s3.meta.client.download_file('mybucket', 'hello.txt', '/tmp/hello.txt')

Similar behavior as S3Transfer's download_file() method,
except that parameters are capitalized. Detailed examples can be found at
:ref:`S3Transfer's Usage <ref_s3transfer_usage>`.

:type Bucket: str
:param Bucket: The name of the bucket to download from.

:type Key: str
:param Key: The name of the key to download from.

:type Filename: str
:param Filename: The path to the file to download to.

:type ExtraArgs: dict
:param ExtraArgs: Extra arguments that may be passed to the
    client operation.

:type Callback: function
:param Callback: A method which takes a number of bytes transferred to
    be periodically called during the download.

:type Config: boto3.s3.transfer.TransferConfig
:param Config: The transfer conf